In [34]:
import torch

In [35]:
from transformers import MarianMTModel, MarianTokenizer

In [36]:
en_ROMANCE_model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
en_ROMANCE_tokenizer = MarianTokenizer.from_pretrained(en_ROMANCE_model_name)
print(en_ROMANCE_tokenizer.supported_language_codes)

['>>fr<<', '>>es<<', '>>it<<', '>>pt<<', '>>pt_br<<', '>>ro<<', '>>ca<<', '>>gl<<', '>>pt_BR<<', '>>la<<', '>>wa<<', '>>fur<<', '>>oc<<', '>>fr_CA<<', '>>sc<<', '>>es_ES<<', '>>es_MX<<', '>>es_AR<<', '>>es_PR<<', '>>es_UY<<', '>>es_CL<<', '>>es_CO<<', '>>es_CR<<', '>>es_GT<<', '>>es_HN<<', '>>es_NI<<', '>>es_PA<<', '>>es_PE<<', '>>es_VE<<', '>>es_DO<<', '>>es_EC<<', '>>es_SV<<', '>>an<<', '>>pt_PT<<', '>>frp<<', '>>lad<<', '>>vec<<', '>>fr_FR<<', '>>co<<', '>>it_IT<<', '>>lld<<', '>>lij<<', '>>lmo<<', '>>nap<<', '>>rm<<', '>>scn<<', '>>mwl<<']


In [37]:
en_ROMANCE = MarianMTModel.from_pretrained(en_ROMANCE_model_name)

In [38]:
ROMANCE_en_model_name = 'Helsinki-NLP/opus-mt-ROMANCE-en'
ROMANCE_en_tokenizer = MarianTokenizer.from_pretrained(ROMANCE_en_model_name)


In [39]:
ROMANCE_en = MarianMTModel.from_pretrained(ROMANCE_en_model_name)

In [42]:
skip = "true"
english = "It was a dark and stormy night"
start = "The"

#english only- spanish behind the scenes
if skip == "true":
    tokenizer = ROMANCE_en_tokenizer
    model = ROMANCE_en

#show spanish
else:
    tokenizer = en_ROMANCE_tokenizer
    model = en_ROMANCE

starting_word = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(start))
prefix = torch.LongTensor(starting_word)

batch = tokenizer.prepare_translation_batch([">>es<<" + english])
english_encoded = model.get_encoder()(**batch)
decoder_start_token = model.config.decoder_start_token_id
# pylint: disable=E1101
partial_decode = torch.LongTensor([decoder_start_token]).unsqueeze(0)
past = (english_encoded, None)
# pylint: enable=E1101

num_tokens_generated = 0

MAX_LENGTH = 100

prediction_list = []
total = 0
#stop when </s> token generated, or max num tokens exceded (just in case)
while True:#next_token_to_add.item() != 0 and x < MAX_LENGTH:
    model_inputs = model.prepare_inputs_for_generation(
    partial_decode, past=past, attention_mask=batch['attention_mask'], use_cache=model.config.use_cache
    )
    with torch.no_grad():
        model_outputs = model(**model_inputs)

    next_token_logits = model_outputs[0][0, -1, :]
    past = model_outputs[1]

    #start with user inputted beginning
    if x < len(prefix):
        next_token_to_add = prefix[x]
    else:
        next_token_to_add = next_token_logits[0].topk(100).indices[1].item()
    
    next_token_logprobs = next_token_logits - next_token_logits.logsumexp()
    score = next_token_logprobs[next_token_to_add].item()
    total += score
    
    #     P(this is cool) =       P(this) * P(is | this) * P(cool | this is)
    # log P(this is cool) = log ( P(this) * P(is | this) * P(cool | this is) )
    #                     = log P(this) + log P(is | this) + log P(cool | this is)
    
    # score_A = -5     exp(score_A) <- positive
    # score_B = 2      exp(score_B)
    # 
    #     P(A) = exp(score_A) / ( exp(score_A) + exp(score_B) )
    # log P(A) = score_A - log ( exp(score_A) + exp(score_B) ) )
        
    #append top 10 predictions for each token to list
    decoded_predictions = []
    for tok in next_token_logits[0].topk(10).indices:
        decoded_predictions.append(tokenizer.convert_ids_to_tokens(tok.item()).replace('\u2581', '\u00a0'))

    #list of lists of predictions
    prediction_list.append(decoded_predictions)

    #add new token to tokens so far
    partial_decode = torch.cat((partial_decode, next_token_to_add.unsqueeze(0).unsqueeze(0)), -1)
    x += 1
    
    if next_token_to_add.item() == 0 or not (num_tokens_generated < MAX_LEN):
        break

#list of tokens used to display sentence
decoded_tokens = [sub.replace('\u2581', '\u00a0') for sub in tokenizer.convert_ids_to_tokens(partial_decode[0])]
decoded_tokens.remove("<pad>")

final = tokenizer.decode(partial_decode[0]).replace("<pad>", '')
print(final)
total/len(decoded_tokens)

NameError: name 'x' is not defined

In [46]:
next_token_logits.argmax()

tensor(114)

In [47]:
next_token_logits[next_token_logits.argmax()]

tensor(9.9219)

In [4]:
import torch

In [32]:
logits = torch.Tensor([-2., 2.]) - 2.0181
logits

tensor([-4.0181, -0.0181])

In [33]:
unnormalized_probs = logits.exp() 
unnormalized_probs

tensor([0.0180, 0.9821])

In [23]:
probs = unnormalized_probs / unnormalized_probs.sum()

In [24]:
logprobs = probs.log()
logprobs

tensor([-4.0181, -0.0181])

In [ ]:
logits - log(sum(exp(logits)))

In [31]:
logits.logsumexp(0)

tensor(2.0181)

In [67]:
save[0].topk(11).indices[10]

tensor(42)

In [42]:
next_token_logits[0].topk(10).values

tensor([9.9219, 8.8910, 8.1260, 6.8402, 6.3231, 5.9794, 5.7846, 5.7819, 5.6579,
        5.5030])

In [13]:
next_token_logits[0].topk(10)

torch.return_types.topk(
values=tensor([9.6353, 8.0945, 5.6484, 5.5065, 4.4912, 4.0338, 3.6761, 3.4046, 3.3394,
        3.0043]),
indices=tensor([ 0,  3,  2, 78, 65, 39, 10, 17, 35, 23]))

In [56]:
next_token_logits[0].topk(11).indices

tensor([  0, 114,  42,  20,   3,  23, 222, 229, 343, 111, 176])

In [60]:
tokenizer.convert_ids_to_tokens([42])

['▁The']

In [21]:
decoded_tokens

['\xa0O', 'cto', 'pus', '\xa0is', '\xa0nice', '</s>']

In [7]:


tokenizer = en_ROMANCE_tokenizer
model = en_ROMANCE
#tokenizer = ROMANCE_en_tokenizer
#model = ROMANCE_en


english = ">>es<<It was a dark and stormy night"
batch = tokenizer.prepare_translation_batch([english])
english_encoded = model.get_encoder()(**batch)
decoder_start_token = model.config.decoder_start_token_id

starting_word = tokenizer.convert_tokens_to_ids(tokenizer.tokenize("La"))
#starting_word = tokenizer.encode("A la")

partial_decode = torch.LongTensor([decoder_start_token]).unsqueeze(0)
past = (english_encoded, None)

prefix = torch.LongTensor(starting_word)
#prefix = torch.LongTensor(tokenizer.convert_tokens_to_ids("A la".replace(' ', '▁').split('▁')))
#prefix = torch.LongTensor(tokenizer.encode("A la"))
#prefix = torch.LongTensor(tokenizer.encode("A▁la▁"))
next_token_to_add = torch.tensor(1)
x = 0

prediction_list = []

while next_token_to_add.item() != 0 and x < 100:
    print(partial_decode)
    model_inputs = model.prepare_inputs_for_generation(
    partial_decode, past=past, attention_mask=batch['attention_mask'], use_cache=model.config.use_cache
    )
    with torch.no_grad():
        model_outputs = model(**model_inputs)

    next_token_logits = model_outputs[0][:, -1, :]
    past = model_outputs[1]
    
    if x < len(prefix):
        next_token_to_add = prefix[x]
        print(x, ": ", next_token_to_add)
    else:
        next_token_to_add = next_token_logits[0].argmax()

    decoded_predictions = []
    for tok in next_token_logits[0].topk(5).indices:
        decoded_predictions.append(tokenizer.convert_ids_to_tokens(tok.item()).replace('\u2581', '\u00a0'))
    
    prediction_list.append(decoded_predictions)
    
    partial_decode = torch.cat((partial_decode, next_token_to_add.unsqueeze(0).unsqueeze(0)), -1)
    x += 1
    
decoded_tokens = tokenizer.convert_ids_to_tokens(partial_decode[0])

decoded_tokens = [sub.replace('\u2581', '\u00a0') for sub in tokenizer.convert_ids_to_tokens(partial_decode[0])] 

final = tokenizer.decode(partial_decode[0]).split("<pad>")[1]
print(final)
print(decoded_tokens)
print(prediction_list)

batch2 = ROMANCE_en_tokenizer.prepare_translation_batch([">>en<< " + final])
spanish_to_english = ROMANCE_en.generate(**batch2)
new_english = ROMANCE_en_tokenizer.decode(spanish_to_english[0]).split("<pad>")[1]

new_english

tensor([[65000]])
0 :  tensor(100)
tensor([[65000,   100]])
tensor([[65000,   100,  4997]])
tensor([[65000,   100,  4997,   531]])
tensor([[65000,   100,  4997,   531,   231]])
tensor([[65000,   100,  4997,   531,   231, 11023]])
tensor([[65000,   100,  4997,   531,   231, 11023,    32]])
tensor([[65000,   100,  4997,   531,   231, 11023,    32,   805]])
tensor([[65000,   100,  4997,   531,   231, 11023,    32,   805,  6386]])
tensor([[65000,   100,  4997,   531,   231, 11023,    32,   805,  6386,   570]])
tensor([[65000,   100,  4997,   531,   231, 11023,    32,   805,  6386,   570,
          3301]])
tensor([[65000,   100,  4997,   531,   231, 11023,    32,   805,  6386,   570,
          3301,     3]])
 La noche era oscura y tempestuosa.
['<pad>', '\xa0La', '\xa0noche', '\xa0era', '\xa0os', 'cura', '\xa0y', '\xa0tem', 'pes', 'tu', 'osa', '.', '</s>']
[['\xa0Era', '\xa0Fue', '\xa0Es', '\xa0Ha', '\xa0Una'], ['\xa0noche', '\xa0tarde', '\xa0oscuridad', '\xa0verdad', '\xa0vela'], ['\xa0era

' The night was dark and stormy.'

In [8]:
import random

In [15]:
tokenizer = en_ROMANCE_tokenizer
model = en_ROMANCE

english = ">>es<<It was a dark and stormy night"
batch = tokenizer.prepare_translation_batch([english])
english_encoded = model.get_encoder()(**batch)
decoder_start_token = model.config.decoder_start_token_id

starting_word = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(""))
prefix = torch.LongTensor(starting_word)
prediction_list = []
alternatives = []
for x in range(0, 5):

    past = (english_encoded, None)
   
    partial_decode = torch.LongTensor([decoder_start_token]).unsqueeze(0)
    y = 0
    next_token_to_add = torch.tensor(1)
    while next_token_to_add.item() != 0 and y < 100:    
        model_inputs = model.prepare_inputs_for_generation(
        partial_decode, past=past, attention_mask=batch['attention_mask'], use_cache=model.config.use_cache
        )
        with torch.no_grad():
            model_outputs = model(**model_inputs)
        next_token_logits = model_outputs[0][:, -1, :]
        past = model_outputs[1]

        if y < len(prefix):
            next_token_to_add = prefix[y]
        else:
            next_token_to_add = next_token_logits[0].topk(4).indices[random.randint(0, 1)]
        decoded_predictions = []
        for tok in next_token_logits[0].topk(5).indices:
            decoded_predictions.append(tokenizer.convert_ids_to_tokens(tok.item()).replace('\u2581', '\u00a0'))

        prediction_list.append(decoded_predictions)
        partial_decode = torch.cat((partial_decode, next_token_to_add.unsqueeze(0).unsqueeze(0)), -1)
        y += 1

    decoded_tokens = tokenizer.convert_ids_to_tokens(partial_decode[0])

    final = tokenizer.decode(partial_decode[0]).split("<pad>")[1]
    print(final)
    alternatives.append(final)
    
alternatives

 Era un noche oscureciÃ¡n y tormenta........
 Fue una oscuridad y una noche tempestuoso
 Fue una noche oscurecida y tormenta.
 Era un noche oscurecida, tempestuoso
 Era una oscuridad, y una noche tempunática


[' Era un noche oscureciÃ¡n y tormenta........',
 ' Fue una oscuridad y una noche tempestuoso',
 ' Fue una noche oscurecida y tormenta.',
 ' Era un noche oscurecida, tempestuoso',
 ' Era una oscuridad, y una noche tempunática']

In [166]:
decoded_tokens

['\xa0',
 '.',
 '\xa0I',
 '\xa0believe',
 '\xa0this',
 '\xa0is',
 '\xa0the',
 '\xa0wrong',
 '\xa0store',
 '.',
 '</s>']

In [35]:
tokenizer = en_ROMANCE_tokenizer
model = en_ROMANCE

english = ">>es<<I think maybe this is the wrong store."
batch = tokenizer.prepare_translation_batch([english])
                                            
eng_to_spanish = model.generate(num_beams=50, num_return_sequences=50, **batch)

translations = []
for x in range(0, 50):
    translations.append(tokenizer.decode(eng_to_spanish[x]).split("<pad>")[1])

translations

[' Creo que esta es la tienda equivocada.',
 ' Creo que tal vez esta es la tienda equivocada.',
 ' Creo que quizás esta es la tienda equivocada.',
 ' Creo que puede que esta sea la tienda equivocada.',
 ' Creo que esta tal vez es la tienda equivocada.',
 ' Creo que quizá esta es la tienda equivocada.',
 ' Creo que ésta es la tienda equivocada.',
 ' Creo que es la tienda equivocada.',
 ' Creo que esta puede ser la tienda equivocada.',
 ' Creo que quizá esta sea la tienda equivocada.',
 ' Creo que quizás esta sea la tienda equivocada.',
 ' Creo que tal vez esta sea la tienda equivocada.',
 ' Pienso que esta es la tienda equivocada.',
 ' Creo que a lo mejor esta es la tienda equivocada.',
 ' Creo que puede que esta es la tienda equivocada.',
 ' Creo que tal vez ésta es la tienda equivocada.',
 ' Creo que esta es una tienda equivocada.',
 ' Creo que puede que ésta sea la tienda equivocada.',
 ' Creo que esto es la tienda equivocada.',
 ' Creo que esta tal vez sea la tienda equivocada.',
 '

In [137]:
back_to_english[0]

tensor([65000,    42,  7025,    26,  2910,     3,     0, 65000, 65000, 65000,
        65000, 65000, 65000, 65000])

In [144]:
ROMANCE_en_tokenizer.decode(back_to_english[0])

"<pad> You're my best friend.<pad><pad><pad><pad><pad><pad><pad><pad>"

In [145]:
">>en<<" + tokenizer.decode(spanish[0]).replace("<pad> ", '')

'>>en<<Eres mi mejor amigo.'

In [147]:
back_to_english[0]

tensor([65000,   111,     7,    66,   148,   660,  2203,     3,     0, 65000,
        65000, 65000, 65000, 65000, 65000, 65000, 65000])

In [155]:
ROMANCE_en_tokenizer.decode([65000,   4932,    7,    66,   148,   660,  2203,     3,     0, 65000,
        65000, 65000, 65000, 65000, 65000, 65000, 65000])

"<pad>You're my best friend.<pad><pad><pad><pad><pad><pad><pad><pad>"

In [164]:
tokenizer = en_ROMANCE_tokenizer
model = en_ROMANCE

english = ">>es<<You're my soulmate"
batch = tokenizer.prepare_translation_batch([english])

spanish = model.generate(**batch)

batch2 = ROMANCE_en_tokenizer.prepare_translation_batch([">>en<<" + tokenizer.decode(spanish[0])])

back_to_english = ROMANCE_en.generate(num_beams=20, num_return_sequences=20, bad_words_ids=[[1695],[1973],[7927],[55],[23],[367],[51],[12390],[1172],[45351],[39]], **batch2)

translations = []
for x in range(0, 20):
    print(back_to_english[x])
    translations.append(ROMANCE_en_tokenizer.decode(back_to_english[x]).replace("<pad>", ''))
    
translations

tensor([65000,   111,    57,   148,  7112,  9036,     0, 65000, 65000, 65000,
        65000, 65000, 65000, 65000, 65000, 65000])
tensor([65000,   111,     7,    66,   148,  7112,  9036,     3,     0, 65000,
        65000, 65000, 65000, 65000, 65000, 65000])
tensor([65000,   111,    57,   148,  7112,  9036,     3,     0, 65000, 65000,
        65000, 65000, 65000, 65000, 65000, 65000])
tensor([65000,   111,     7,    66,   148,  7112,  9036,     0, 65000, 65000,
        65000, 65000, 65000, 65000, 65000, 65000])
tensor([65000,   111,     7,    66,   148,  7112,  9036,     3,   111,     7,
           66,   148,  7112,  9036,     3,     0])
tensor([65000,    31,     7,    66,   148,  7112,  9036,     3,     0, 65000,
        65000, 65000, 65000, 65000, 65000, 65000])
tensor([65000,    31,    57,   148,  7112,  9036,     0, 65000, 65000, 65000,
        65000, 65000, 65000, 65000, 65000, 65000])
tensor([65000,    31,     7,    66,   148,  7112,  9036,     0, 65000, 65000,
        65000, 6500

[' You are my soul mate',
 " You're my soul mate.",
 ' You are my soul mate.',
 " You're my soul mate",
 " You're my soul mate. You're my soul mate.",
 " you're my soul mate.",
 ' you are my soul mate',
 " you're my soul mate",
 " You're my soul mate! You're my soul mate!",
 " You're my soulmate.",
 " you're my soul mate. You're my soul mate.",
 " You know, you're my soul mate.",
 " You're my soul mate. You are my soul mate.",
 " You're my soul mate, you're my soul mate.",
 " You're my soulmate. You're my soulmate.",
 " You're my soul mate!",
 ' you are my soul mate.',
 " You're my soul mate. You're my soulmate.",
 " You're my soul mate. You're my soul mate!",
 ' You are my soulmate.']

In [159]:
model.generate??

In [113]:
bad_words_ids = [ROMANCE_en_tokenizer.encode(bad_word) for bad_word in ['-', '"']]
bad_words_ids

[[23, 0], [51, 0]]

In [165]:
ROMANCE_en_tokenizer.convert_ids_to_tokens([0])

['</s>']

In [162]:
ROMANCE_en_tokenizer.convert_tokens_to_ids('♪')

47075

In [57]:
english = ">>es<<I think maybe this is the wrong store."
batch = tokenizer.prepare_translation_batch([english])
                                            
spanish = model.generate(**batch)

batch = ROMANCE_en_tokenizer.prepare_translation_batch([">>en<<" + tokenizer.decode(spanish[0]).replace("<pad>", '')])
back_to_english = ROMANCE_en.generate(**batch)
machine_translation = ROMANCE_en_tokenizer.decode(back_to_english[0]).replace("<pad>", '')

spanish_encoded = ROMANCE_en.get_encoder()(**batch)
decoder_start_token = ROMANCE_en.config.decoder_start_token_id
# pylint: disable=E1101
partial_decode = torch.LongTensor([decoder_start_token]).unsqueeze(0)
past = (spanish_encoded, None)
# pylint: enable=E1101
next_token_to_add = torch.tensor(1)
x = 0

prediction_list = []

while next_token_to_add.item() != 0 and x < 100:
    model_inputs = ROMANCE_en.prepare_inputs_for_generation(
    partial_decode, past=past, attention_mask=batch['attention_mask'], use_cache=ROMANCE_en.config.use_cache
    )
    with torch.no_grad():
        model_outputs = ROMANCE_en(**model_inputs)

    next_token_logits = model_outputs[0][:, -1, :]
    past = model_outputs[1]
   
    if x == 3:
        next_token_to_add = next_token_logits[0].topk(4).indices[1]
    else:
        next_token_to_add= next_token_logits[0].argmax()
    decoded_predictions = []
    for tok in next_token_logits[0].topk(10).indices:
        decoded_predictions.append(ROMANCE_en_tokenizer.convert_ids_to_tokens(tok.item()).replace('\u2581', '\u00a0'))

    prediction_list.append(decoded_predictions)

    partial_decode = torch.cat((partial_decode, next_token_to_add.unsqueeze(0).unsqueeze(0)), -1)
    x+= 1

decoded_tokens = [sub.replace('\u2581', '\u00a0') for sub in ROMANCE_en_tokenizer.convert_ids_to_tokens(partial_decode[0])]
decoded_tokens.remove("<pad>")
final = ROMANCE_en_tokenizer.decode(partial_decode[0]).split("<pad>")[1]

final

'. I believe this is the wrong store.'

In [40]:
machine_translation

' Creo que esta es la tienda equivocada.'

In [24]:
model.generate??

In [11]:
next_token_logits[0].topk(3).indices[1]

tensor(3)